In [1]:
from heat_pump_model import heat_pump
from libraries import * 
from refrigerant_properties import *
import numpy as np
from utilities.unit_defs import Q_

In [10]:
# function for cascade cycle
class cascade_heat_pump(heat_pump):

    def __init__(self):

        # number of stages:
        self.n_stages = 3

        # delta T between the two fluids (both with constant temperature) on each side of the  heat exchangers
        self.cascade_HE_dT = Q_(5, 'degK')
        # delta T between the 
        self.hot_buffer = Q_(5, 'degK')
        self.cold_buffer = Q_(5, 'degK')

        # specified refrigerants: initialize with the same refrigerant in all cycles
        self.refrigerant_list = ['ammonia']*self.n_stages
        
    def calculate_COP(self):

        # calculate intermediate temperature levels based on the number of cycles:
        T_levels_center = np.linspace(self.cold_temperature_available - 5, 
                                      self.hot_temperature_desired + 5, 
                                      self.n_stages + 1)
        # initialize temperature levels, rows represent different stages, columns represent evaporator and condenser temperatures
        T_levels = np.zeros( (self.n_stages, 2) )
        for i in range(self.n_stages):
            # first stage, evaporator dT already accounted for
            if i == 0:
                # evaporator temperature
                T_levels[i][0] = T_levels_center[i] 
                # condenser temperature
                T_levels[i][1] = T_levels_center[i+1] + self.cascade_HE_dT/2
            # last stage, condenser dT already accounted for
            elif i == self.n_stages - 1:
                # evaporator temperature
                T_levels[i][0] = T_levels_center[i] - self.cascade_HE_dT/2
                # condenser temperature
                T_levels[i][1] = T_levels_center[i+1] 
            # remaining stages
            else: 
                # evaporator temperature
                T_levels[i][0] = T_levels_center[i] - self.cascade_HE_dT/2
                # condenser temperature
                T_levels[i][1] = T_levels_center[i+1] + self.cascade_HE_dT/2

        # run HP model for each stage
        H_matrix = np.zeros( (self.n_stages, 4) )  # enthalpies at points 1:4 for each stage
        for i in range(self.n_stages):
            # generate HP model
            hp = heat_pump()
            hp.refrigerant_flag = True
            hp.carnot_efficiency_factor_flag = False  # redundant?
            hp.refrigerant = self.refrigerant_list[i]
            hp.cold_temperature_available = T_levels[i][0]
            hp.hot_temperature_desired = T_levels[i][1]
            hp.hot_buffer = 0
            hp.cold_buffer = 0

            # run HP, calculate enthalpies
            hp.calculate_COP() 
            H_matrix[i][0] = hp.H_1
            H_matrix[i][1] = hp.H_2
            H_matrix[i][2] = hp.H_3
            H_matrix[i][3] = hp.H_4

        '''calculate overall COP'''

        # heat rejection from the uppermost stage
        Q_rej_permass = H_matrix[-1][1] - H_matrix[-1][2]

        # calculate flow ratios between adjacent stages (top stage mass flowrate over bottom stage mass flow rate, calculated from a heat balance)
        flow_ratio = np.zeros(self.n_stages-1)
        for i in range(self.n_stages)-1:
            flow_ratio[i] = (H_matrix[i+1][0] - H_matrix[i+1][3]) / (H_matrix[i][1] - H_matrix[i][2])

        # calculate denominator of COP: product of work_permass and product of flow ratios starting from the stage and going upwards 
        # example for 3 stages: COP = Qrej / (W1*(m1/m2)*(m2/m3) + W2*(m2/m3) + W3)
        denominator = 0
        for i in range(self.n_stages):
            work_i_permass = H_matrix[i][1] - H_matrix[i][0]
            denominator += work_i_permass * np.prod(flow_ratio[i:])

        self.COP = Q_rej_permass / denominator

In [11]:
# intialize class instance
hp = cascade_heat_pump()

# print default properties
print(hp.n_stages)
print(hp.cascade_HE_dT)
print(hp.hot_buffer)
print(hp.cold_buffer)
print(hp.refrigerant_list)

# set fixed parameters
hp.cold_temperature_available = 55
hp.hot_temperature_desired = 5

# calculate cascade HP COP
hp.calculate_COP()

3
5 kelvin
5 kelvin
5 kelvin
['ammonia', 'ammonia', 'ammonia']


DimensionalityError: Cannot convert from 'dimensionless' to 'kelvin'